# Svelty
A `Svelt Stores` implementation in Python.
See `Svelt Stores` [documentation](https://svelte.dev/docs#run-time-svelte-store) for more information on the original.

## Setup

In [ ]:
import nbdev

In [ ]:
#| default_exp store

In [ ]:
#|export
from __future__ import annotations
from typing import Callable, TypeVar,  Generic, Union, Optional, Set, Protocol

## Svelte Store contract

1. A store must contain a `.subscribe` method, which must accept as its argument a `subscription function`(aka Subscriber or Callback). This `subscription function` must be immediately and synchronously called with the store's current value upon calling `subscribe`. All of a store's active subscription functions must later be synchronously called whenever the store's value changes.

1. The `.subscribe` method must return an `unsubscribe function`(aka Unsubscriber). Calling an `unsubscribe function` must `stop` its subscription, and its corresponding `subscription function` must not be called again by the store.

1. A store may optionally contain a `.set` method, which must accept as its argument a new value for the store, and which synchronously calls all of the store's active subscription functions. Such a store is called a writable store.


~~For interoperability with RxJS Observables, the .subscribe method is also allowed to return an object with an .unsubscribe method, rather than return the unsubscription function directly. Note however that unless .subscribe synchronously calls the subscription (which is not required by the Observable spec), Svelte will see the value of the store as undefined until it does.~~

[Store Contract Documentation](https://svelte.dev/docs#component-format-script-4-prefix-stores-with-$-to-access-their-values-store-contract)

### Types Definition

In [ ]:
#| export

T = TypeVar("T")
covT = TypeVar("covT", covariant=True)
Subscriber = Callable[[T], None] # a callback
Unsubscriber = Callable[[], None] # a callback to be used upon termination of the subscription
 
class StoreProtocol(Protocol, Generic[covT]):
    def subscribe(self, subscriber: Subscriber[T]) -> Unsubscriber: ...

## Readable and Writable Stores

### Definition

In [ ]:
#|export
class Base:  #see SimpleNamespace: https://docs.python.org/3/library/types.html
    def __init__(self, /, **kwargs):
        self.__dict__.update(kwargs)

Updater = Callable[[T], T]

class Store(Base, StoreProtocol[T]):
    value: T
    subscribers: Set[Subscriber]
    def subscribe(self, callback: Subscriber) -> Unsubscriber:
        return lambda: None
    def get(self) -> T: return self.value
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.get()!r})"

In [ ]:
class Readable(Store[T]): pass

class Writable(Store[T]):
    set: Subscriber
    update: Optional[Callable[[Updater],None]] = None

### Writable Stores

In [ ]:
class Writable(Store[T]):
    def __init__(self, initial_value: T) -> None:
        self.value: T = initial_value
        self.subscribers: Set[Subscriber] = set()

    def subscribe(self, callback: Subscriber) -> Unsubscriber:
        self.subscribers.add(callback)
        callback(self.value)

        def unsubscribe() -> None:
            self.subscribers.remove(callback) if callback in self.subscribers else None
        return unsubscribe
    
    def set(self, new_value: T) -> None:
        if new_value != self.value:
            self.value = new_value
            for subscriber in self.subscribers:
                subscriber(new_value)
    
    def update(self, fn: Callable[[T], T]) -> None:
        self._set(fn(self.value))
    
    def __len__(self) -> int:
        return len(self.subscribers)

### A `Writable` in action

Let's create a `Writable`.  Remember that it returns an unsubscriber function.

In [ ]:
a = Writable(1)
a

In [ ]:
u1 = a.subscribe(lambda x: print("1:",x))

In [ ]:
a

A `writable` can be set from the outside. When it happens, all its subscribers will react.

In [ ]:
a.set(2)

After unsubscribing, the subscriber should not be called anymore:

In [ ]:
u1()
a.set(3)

If you try to unsubscibe twice, it won't break.  It just does nothing the second time.

In [ ]:
u1()

In [ ]:
u1(),u1(), a

### Readable store


A`Readable` is a `Writable` with protected `set` and `update` methods.

In [ ]:
class Readable(Writable): 
    def set(self, *args, **kwargs): raise Exception("Cannot set a Readable Store.")
    def update(self, *args, **kwargs): raise Exception("Cannot update a Readable Store.")


In [ ]:
b = Readable(10)

In [ ]:
u = b.subscribe(lambda x: print("2:",x))

We can subscribe to our `readable`, but nothing happens, we cannot `set` a `Readable` from the outside.

In [ ]:
try:
    b.set("bar") # should fail
except Exception as error:
  print(error)


A `store` that does not change is not useful. A `Readable` is like a `writable` where there is only one "thing" from the outside that can change its value. Lets change `writable` to add this "thing", which we will call a `Notifier`.

In [ ]:
Notifier = Callable[[Subscriber], Union[Unsubscriber, None]]

In [ ]:
#|export
class Writable(Store[T]):
    def __init__(self, initial_value: T, start: Notifier=lambda x: None) -> None:
        self.value: T = initial_value
        self.subscribers: Set[Subscriber] = set()
        self.stop: Optional[Unsubscriber] = None
        self.start: Notifier = start

    def subscribe(self, callback: Subscriber) -> Unsubscriber:
        self.subscribers.add(callback)
        if (len(self.subscribers) == 1):
            self.stop = self.start(callback) or (lambda: None)
        callback(self.value)

        def unsubscribe() -> None:
            self.subscribers.remove(callback) if callback in self.subscribers else None
            if (len(self.subscribers) == 0):
                self.stop() if self.stop else None
                self.stop = None
        return unsubscribe
    
    def set(self, new_value: T) -> None:
        if new_value == self.value: return None
        self.value = new_value
        if not self.stop: return None # no subscribers
        for subscriber in self.subscribers:
            subscriber(new_value)
    
    def update(self, fn: Callable[[T], T]) -> None:
        self.set(fn(self.value))
    
    def __len__(self) -> int:
        return len(self.subscribers)

The previous functionality still works.

In [ ]:
del a

In [ ]:
a = Writable(1)
u1 = a.subscribe(lambda x: print("1:",x))
u2 = a.subscribe(lambda x: print("2:",x))

In [ ]:
u1(), u1()

In [ ]:
a.set(3)

In [ ]:
u2(), u2()

In [ ]:
a.set(4)

In [ ]:
a.get()

In [ ]:
u1 = a.subscribe(lambda x: print("1:",x))
u2 = a.subscribe(lambda x: print("2:",x))

In [ ]:
u1(), u2(), a.get()

In [ ]:
y = lambda x: x
b = y(a)

In [ ]:
a, b

But now, we can start the store with a `Notifier` that asynchronously set the value of the store from the outside.

Let's test by creating an asynchronous notifier.

In [ ]:
from threading import Event, Thread

In [ ]:
def every(interval, func, *args):
    stopped = Event()
    def loop():
        while not stopped.wait(interval): # the first call is in `interval` secs
            func(*args)
    Thread(target=loop).start()    
    return stopped.set

In [ ]:
def start(set): # notifier
    count = 0
    def incrementCounter():
        nonlocal count
        count = count +1
        set(count)
    cancel = every(1, incrementCounter)
    return cancel

In [ ]:
def myset(x):
    value = x
    print("myset:", value)

In [ ]:
stop = start(myset)

In [ ]:
import time

In [ ]:

time.sleep(3)
stop()

In [ ]:
b = Writable(0, start)

In [ ]:
b

In [ ]:
u1 = b.subscribe(lambda x: print("1:",x))

In [ ]:
time.sleep(4)
u1()

Nice, it works. Now, let's fix Readable.

In [ ]:
#|export
class Readable(Writable[T]): 
    def __init__(self, initial_value: T, start: Notifier) -> None:
        super().__init__(initial_value, start)
    def set(self, *args, **kwargs): raise Exception("Cannot set a Readable Store.")
    def update(self, *args, **kwargs): raise Exception("Cannot update a Readable Store.")

Now, we need to provide a `Notifier` to create a `Readable` store:

In [ ]:
try:
    c = Readable(0) # shoud fail
except Exception as error:
    print(error)

### A `Readable` in action

In [ ]:
c = Readable(0, start)
c

Notice that while there is no subscribers, the `Notifier` is not started.

In [ ]:
time.sleep(3)
c

The first subscriber starts the `Notifier`.

In [ ]:
stop = c.subscribe(lambda x: print("1:",x))

In [ ]:
time.sleep(3)
stop()

## Derived Store

A `Derived Store` takes a store and transforms it in another store.

In [ ]:
#|export
class Derived(Writable):
    def __init__(self, source: Store, fn: Updater) -> None:
        self.target = Writable(source.get())
        self.start: Notifier = lambda x: self.target.set(fn(x))
        self.stop = source.subscribe(self.start)
    def get(self) -> T: return self.target.get()
    def set(self, *args, **kwargs): raise Exception("Cannot set a Derived Store.")
    def update(self, *args, **kwargs): raise Exception("Cannot update a Derived Store.")
    def subscribe(self, callback: Subscriber) -> Unsubscriber:
        return self.target.subscribe(callback)

### A `Derived` in action

In [ ]:
a = Writable(1)
u1 = a.subscribe(lambda x: print("1:",x))

In [ ]:
b = Derived(a, lambda x: x*2)

In [ ]:
a,b

In [ ]:
a, b.get()

In [ ]:
a.set(2), a,b

In [ ]:
u1 = b.subscribe(lambda x: print("2:",x))

In [ ]:
a.set(42)

In [ ]:
user = Writable({"name": "John", "age": 42})
user

In [ ]:
user.update(lambda x: x | {"age": 21})
user

In [ ]:
u1 = user.subscribe(lambda x: print("user_subscriber_1:",x))

In [ ]:
user.update(lambda x: x | {"age": 42})

In [ ]:
name = Derived(user, lambda x: x["name"])
name

In [ ]:
u2 = name.subscribe(lambda x: print("name_subscriber_1:",x))

In [ ]:
user.update(lambda x: x | {"age": 56})

In [ ]:
user.update(lambda x: x | {"name": "Fred"})

## Export

In [ ]:
nbdev.nbdev_export()